In [1]:
import os
import warnings

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Set `PATH` to include the directory containing saved_model_cli
PATH = %env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: PATH=/home/jupyter/.local/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [2]:
import datetime
import shutil

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_text as text


#from official.nlp import optimization
tf.get_logger().setLevel("ERROR")

In [3]:
!git clone https://github.com/eminedemirbas/Hackathon-YZTA

Cloning into 'Hackathon-YZTA'...
remote: Enumerating objects: 4636, done.
remote: Counting objects: 100% (4636/4636), done.
remote: Compressing objects: 100% (4600/4600), done.
remote: Total 4636 (delta 80), reused 4485 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (4636/4636), 4.31 MiB | 7.46 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [4]:
repo_path = "/content/Hackathon-YZTA"


data_path = os.path.join(repo_path, 'data')


if os.path.exists(data_path):
    print(f"Data klasörü bulundu: {data_path}")
else:
    print("Data klasörü bulunamadı.")

Data klasörü bulundu: /content/Hackathon-YZTA/data


In [5]:
# Veriyi yüklemek için klasör yolunu kullanacağız
train_dir = os.path.join(data_path, 'train')
test_dir = os.path.join(data_path, 'test')

In [6]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict

def load_data_from_directories(train_dir, test_dir):
    def read_from_category(base_path, category):
        texts, labels = [], []
        for label_name in ["pos", "neg"]:
            label_value = 1 if label_name == "pos" else 0
            folder = os.path.join(base_path, label_name)
            for filename in os.listdir(folder):
                if filename.endswith(".txt"):
                    file_path = os.path.join(folder, filename)
                    with open(file_path, "r", encoding="utf-8") as file:
                        texts.append(file.read())
                        labels.append(label_value)
        return pd.DataFrame({"text": texts, "label": labels})

    train_df = read_from_category(train_dir, "train")
    test_df = read_from_category(test_dir, "test")

    train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
    test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

    dataset = DatasetDict({
        "train": Dataset.from_pandas(train_df),
        "test": Dataset.from_pandas(test_df)
    })

    return dataset

dataset = load_data_from_directories(train_dir, test_dir)

In [7]:
from transformers import AutoTokenizer

model_ckpt = "dbmdz/bert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

In [8]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/3443 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=2)

from transformers import TrainingArguments
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir="./sentiment_results",
    num_train_epochs=3,  # İstediğin epoch sayısını buraya yazabilirsin
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to=None
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
500,0.149000
1000,0.033200


TrainOutput(global_step=1293, training_loss=0.07047509923663742, metrics={'train_runtime': 301.8034, 'train_samples_per_second': 34.224, 'train_steps_per_second': 4.284, 'total_flos': 679418522703360.0, 'train_loss': 0.07047509923663742, 'epoch': 3.0})

In [10]:
# Modeli kaydetme
model.save_pretrained('/content/drive/MyDrive/saved_model')
tokenizer.save_pretrained('/content/drive/MyDrive/saved_model')

('/content/drive/MyDrive/saved_model/tokenizer_config.json',
 '/content/drive/MyDrive/saved_model/special_tokens_map.json',
 '/content/drive/MyDrive/saved_model/vocab.txt',
 '/content/drive/MyDrive/saved_model/added_tokens.json',
 '/content/drive/MyDrive/saved_model/tokenizer.json')

In [11]:
results = trainer.evaluate(tokenized_dataset["test"])

# Sonuçları yazdırma
print(f"Test doğruluğu: {results['eval_accuracy']}")

Test doğruluğu: 0.936


In [18]:
import torch

# Cihazı otomatik belirle (GPU varsa kullan, yoksa CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modeli cihaza taşı
model.to(device)

text = "Bugün karbon izin az çıktı, bilinçlisin!"

# Metni tokenle ve cihaza taşı
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs = {key: val.to(device) for key, val in inputs.items()}

# Tahmin al
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Sonuçları işle
predicted_class_id = logits.argmax(dim=-1).item()
labels = ["neg", "pos"]  # Sıralama senin modeline göre değişebilir

predicted_label = labels[predicted_class_id]
print(f"Metin: {text}")
print(f"Modelin tahmini: {predicted_label}")


Metin: Bugün karbon izin az çıktı, bilinçlisin!
Modelin tahmini: pos
